# Olist E-Commerce Website Data Analysis 

Dataset link: https://www.kaggle.com/olistbr/brazilian-ecommerce

## Table of content:
### Setting up:
* Importing Libraries


* Loading the Datasets and Data Cleaning

### Visualization and Data Insights:



* Boxplot For The Product Weights and Freight Value (Univariable)


* General Exploration Histogram Over All Dataframe Features (Univariable)


* What Is The Most Frequent Payment Type ? (Univariable)


* Payment Value Distribution Per Customer (Univariable)


* Freight Value Vs. Product Weight Scatterplot (Bivariable)


* What Is The Most Frequent Customer State ? (Bivariable)


* What Is The Most In Demand Product Category ? (Bivariable)


* What I The Most Highly Rated Product Category ? (Bivariable)


* what Is The Nicest Customer State ? (Bivariable)


* Payment Vs. Freight value (Bivariable)


* Payment Value Vs. Months vs. Year (Multivariable)


* Correlogram of Olist Master Dataframe (Multivariable)



### Machine Learning Application

* Clustering Customers Based On Review Rating and Purchase Power (Unsupervised Learning)


### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from sklearn.preprocessing import normalize
from sklearn.metrics import r2_score, confusion_matrix
import datetime
import random


%matplotlib inline
sns.set()
pd.options.display.max_columns = None

### Loading the Data

#### customers_dataset

In [ ]:
# no nulls
customers_dataset_df = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv')
customers_dataset_df_clean = customers_dataset_df.copy()

customers_dataset_df.info()

#### geolocation_dataset

In [ ]:
# no nulls
geolocation_dataset_df = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_geolocation_dataset.csv')
geolocation_dataset_df_clean = geolocation_dataset_df.copy()

geolocation_dataset_df.info()

#### order_items

In [ ]:
# no nulls

order_items_df = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv')
order_items_df_clean = order_items_df.copy()

order_items_df.info()

#### order_payments

In [ ]:
# no nulls
order_payments_dataset_df = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv')
order_payments_dataset_df_clean = order_payments_dataset_df.copy()

order_payments_dataset_df.info()

#### order_reviews_dataset

In [ ]:
# review_comment_title and review_comment_message fields are empty which is fine, we will not be using them in out analysis so
# we will drop the columns in addition to review_creation_date and review_answer_timestamp columns
order_reviews_dataset_df = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv')
order_reviews_dataset_df_clean = order_reviews_dataset_df.copy()

order_reviews_dataset_df.info()

In [ ]:
order_reviews_dataset_df_clean.drop(columns = ['review_comment_title','review_comment_message','review_creation_date','review_answer_timestamp'], inplace = True)

In [ ]:
order_reviews_dataset_df_clean.head()

In [ ]:
order_reviews_dataset_df_clean.info()

In [ ]:
order_reviews_dataset_df_clean.describe()

#### orders_dataset

In [ ]:
# 1.we will be assessing if our estimation on the order_estimated_delivery_date is correct most of the time or not, so in this casr
# we will create a df that has only the timestamps and dates and we will be dropping this nan values to do so, that is so we 
# do not loose data for other operations from the original dataframe.
# 2. we need to change  the data type of the dates to datetime.
orders_dataset_df = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv')
orders_dataset_df_clean = orders_dataset_df.copy()
orders_dataset_df.info()


In [ ]:
orders_dataset_df_clean.head()

In [ ]:
# ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
order_datetime_list = ['order_purchase_timestamp', 'order_approved_at',\
                       'order_delivered_carrier_date', 'order_delivered_customer_date',\
                       'order_estimated_delivery_date']


orders_dataset_df_clean[order_datetime_list] = orders_dataset_df_clean[order_datetime_list].apply(pd.to_datetime, errors = 'coerce')
orders_dataset_df_clean.head()

In [ ]:
# testing
orders_dataset_df_clean.info()

In [ ]:
# checking valid values for order_status
orders_dataset_df_clean.order_status.unique()

In [ ]:
# checking if for the delivered items we have the all the timings, the invoiced only can have nan values it is fine for the dates variables.
# we can see that we still have missing dates for already delivered orders, we can investigate closely in the company to see why we are missing these dates.

orders_dataset_df_clean.query("order_status in 'delivered'").isnull().sum()

In [ ]:
# getting the indexes of the columns that have a delivered order_status and a null order_approved_at because it does not make sense
delivered_and_not_approved_indexes = orders_dataset_df_clean[orders_dataset_df_clean['order_approved_at'].isnull()].query("order_status in 'delivered'").index

In [ ]:
orders_dataset_df_clean.drop(inplace = True, index = delivered_and_not_approved_indexes)

In [ ]:
orders_dataset_df_clean.query("order_status in 'delivered'").isnull().sum()

#### products_dataset

In [ ]:
products_dataset_df = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_products_dataset.csv')
products_dataset_df_clean = products_dataset_df.copy()

products_dataset_df.info()

In [ ]:
# we will be dropping the rows that have missing product_category_name, product_name_lenght, product_description_lenght, product_photos_qty
# which are 610 rows of 32951
products_dataset_df_clean[products_dataset_df_clean['product_category_name'].isnull()].shape
null_products_indexes = products_dataset_df_clean[products_dataset_df_clean['product_category_name'].isnull()].index
products_dataset_df_clean.drop(inplace = True, index = null_products_indexes)

In [ ]:
# testing
print(products_dataset_df_clean.shape[0],products_dataset_df.shape[0])

In [ ]:
# 1 row missing dimensions, we will drop it also since it has negligible effect on our analysis
products_dataset_df_clean.info()

In [ ]:
additional_products_missing_g_indexes = products_dataset_df_clean[products_dataset_df_clean['product_weight_g'].isnull()].index

In [ ]:
products_dataset_df_clean.drop(inplace = True, index = additional_products_missing_g_indexes)

In [ ]:
products_dataset_df_clean.info()

In [ ]:
products_dataset_df_clean.describe()

#### sellers_dataset

In [ ]:
# no missing values
sellers_dataset_df = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv')
sellers_dataset_df_clean = sellers_dataset_df.copy()

sellers_dataset_df.info()

#### product_category_name_translation

In [ ]:
product_category_name_translation_df = pd.read_csv('/kaggle/input/brazilian-ecommerce/product_category_name_translation.csv')
product_category_name_translation_df.info()

In [ ]:
# creating a master dataframe, that has all the clean dataframes to get more insights when we want to visualize, model and predict
master_df = orders_dataset_df_clean.copy()

master_df = master_df.merge(customers_dataset_df_clean, on='customer_id', indicator=True)

In [ ]:
master_df = master_df.merge(order_reviews_dataset_df_clean, on='order_id')

In [ ]:
master_df = master_df.merge(order_payments_dataset_df_clean, on='order_id')

In [ ]:
master_df = master_df.merge(order_items_df_clean, on = 'order_id')

In [ ]:
master_df = master_df.merge(products_dataset_df_clean, on = 'product_id')

In [ ]:
master_df = master_df.merge(sellers_dataset_df_clean, on = 'seller_id')

In [ ]:
master_df.shape

# Exploratory Data Visualization

#### In this section we will be answering inquiries using visualizations of data insights using univariable, bivariable and multivariable plots

In [ ]:
master_df.shape

In [ ]:
# we only have two columns having null values, order_delivered_carrier_date and order_delivered_customer_date  
master_df.isnull().sum()

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(master_df.corr().round(2), xticklabels=master_df.corr().columns, yticklabels=master_df.corr().columns, cmap='RdYlGn', center=0, annot=True)

plt.title('Correlogram of Olist Master Dataframe', fontsize=22)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
master_df.corr().round(2)> 0.5

In [ ]:
abs(master_df.corr().round(2))>0.5

In [ ]:
# we can see some correlation between our variables, we will use > 0.5 correlation to get a clearer view in our exploration

plt.figure(figsize=(12,10))
sns.heatmap(abs(master_df.corr().round(2))>0.5 , xticklabels=master_df.corr().columns, yticklabels=master_df.corr().columns, cmap='RdYlGn', center=0, annot=True)

plt.title('Simplified Correlogram of Olist Master Dataframe', fontsize=22)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
list_of_correlated_columns = []

for i in (abs(master_df.corr().round(2))>0.5).columns:
    for j in (abs(master_df.corr().round(2))>0.5).columns:
        if ((abs(master_df.corr().round(2))>0.5).loc[i,j] == True):
            list_of_correlated_columns.append((i,j))

In [ ]:
type(list_of_correlated_columns[0][0])

In [ ]:
# gettng rid of the pairs constructed of the same variables
list_of_correlated_columns_not_same = []

for k in range(len(list_of_correlated_columns)):
    if list_of_correlated_columns[k][0] != list_of_correlated_columns[k][1]:
        list_of_correlated_columns_not_same.append((list_of_correlated_columns[k][0], list_of_correlated_columns[k][1]))


In [ ]:
list_of_correlated_columns_not_same

In [ ]:
# since we have see correlation between the product_weight_g and freight_value in the heatmap we will be drawing
# scatter plot with a regression line.

plt.figure(figsize=(14,8))

product_weight_g_reg = master_df.groupby('customer_id').sum()[['product_weight_g','freight_value']].reset_index()['product_weight_g']
freight_value_reg = master_df.groupby('customer_id').sum()[['product_weight_g','freight_value']].reset_index()['freight_value']
sns.regplot(x=product_weight_g_reg, y=freight_value_reg, scatter_kws={'alpha':0.7, 'color':'skyblue'},\
           line_kws={'color':'red'});

plt.title('Freight Value Vs. Product Weight Scatterplot'.title(), fontsize=22)
plt.xlabel('Product Weight'.title(), fontsize=15)
plt.ylabel('Freight Value'.title(), fontsize=15)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(14,8))

product_weight_g_reg = master_df.groupby('customer_id').sum()[['product_weight_g','freight_value']].reset_index()['product_weight_g']
freight_value_reg = master_df.groupby('customer_id').sum()[['product_weight_g','freight_value']].reset_index()['freight_value']
sns.regplot(x=product_weight_g_reg, y=freight_value_reg, scatter_kws={'alpha':0.7, 'color':'skyblue'},\
           line_kws={'color':'red'});

plt.xlim(0, 200000)

plt.title('Freight Value Vs. Product Weight Scatterplot'+'\n'+'Zoomed on Product Weight Between 0g and 200000g', fontsize=22)
plt.xlabel('Product Weight'.title(), fontsize=15)
plt.ylabel('Freight Value'.title(), fontsize=15)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()


In [ ]:
# alot of outliers for the product weight
plt.figure(figsize=(14,8))
sns.boxplot(x=product_weight_g_reg, color='skyblue');

plt.title('Boxplot For The Product Weights', fontsize=18);
plt.xlabel('Product Weight'.title(), fontsize=12);

In [ ]:
#taking up to the 90% quantile
plt.figure(figsize=(14,8))

sns.boxplot(x=product_weight_g_reg[product_weight_g_reg < product_weight_g_reg.quantile(0.90)], color='skyblue');
plt.title('Boxplot For The Product Weights', fontsize=18);
plt.xlabel('Product Weight'.title(), fontsize=12);

In [ ]:
# altot of outliers in the freight value, we will be taking the 90% quantile
plt.figure(figsize=(14,8))

sns.boxplot(freight_value_reg, color='skyblue');

plt.title('Boxplot For The Freight Value', fontsize=18);
plt.xlabel('Freight Value'.title(), fontsize=12);

In [ ]:
#taking up to the 90% quantile
plt.figure(figsize=(14,8))

sns.boxplot(freight_value_reg[freight_value_reg < freight_value_reg.quantile(0.90)], color='skyblue');

plt.title('Boxplot For The Freight Value', fontsize=18);
plt.xlabel('Freight Value'.title(), fontsize=12);

In [ ]:
weight_freight_reg = pd.DataFrame({'product_weights':product_weight_g_reg, 'freight_value':freight_value_reg})

In [ ]:
weight_freight_reg.iloc[:,1].head()

In [ ]:
# ploting the regresion line with the 90% quantile, we can still see that a linear regression does not fit it well
plt.figure(figsize=(14,8))

quantile_weight_freight_reg = weight_freight_reg[weight_freight_reg < weight_freight_reg.quantile(0.9)]

sns.regplot(quantile_weight_freight_reg['product_weights'] ,quantile_weight_freight_reg['freight_value'],\
            scatter_kws={'alpha':0.2, 'color':'skyblue'},\
            line_kws={'color':'red'});

plt.title('Freight Value Vs. Product Weight Scatterplot'+'\n'+'Using 90% Quantile' , fontsize=18);
plt.ylabel('Freight Value'.title(), fontsize=12);
plt.xlabel('Product Weight'.title(), fontsize=12);

In [ ]:
# general exploration
master_df.hist(figsize=(15,20), color='skyblue');

## Univariable Plots

* Type of payments distribution 
* Orders prices distribution 
* Count plot for states 
* Count plot for products category

In [ ]:
# checking the payment types
master_df.payment_type.unique()

In [ ]:
payment_type_count_plot_df = master_df[['payment_type','customer_id']].groupby(['customer_id', 'payment_type']).count().reset_index()

In [ ]:
# The customers seems to prefer to pay by credit card over other options

plt.figure(figsize=(14,8))

sns.countplot(payment_type_count_plot_df['payment_type']);

plt.title('What is the most frequent payment type?'.title() , fontsize=20);
plt.ylabel('count'.title(), fontsize=14);
plt.xlabel('payment type'.title(), fontsize=14);

In [ ]:
master_df.groupby('customer_id').sum().reset_index()['payment_value'].describe()

In [ ]:
#### the price distribution of the invoices in right skewed as we see below, if we want to display the maximum values we will end up with no insights on the graph.

plt.figure(figsize=(14,8))

min_order_payment_value = min(master_df.groupby('customer_id').sum().reset_index()['payment_value'])
max_order_payment_value = max(master_df.groupby('customer_id').sum().reset_index()['payment_value'])

bins= np.linspace(start=min_order_payment_value, stop=1400, num=50)
plt.hist(x=np.asarray(master_df.groupby('customer_id').sum().reset_index()['payment_value']), bins=bins,\
         color='skyblue');

plt.title('payment value distribution per customer'.title()+'\n'+'Between 0LCU and 1,400LCU' , fontsize=20);
plt.ylabel('count'.title(), fontsize=14, rotation=90);
plt.xlabel('payment value'.title(), fontsize=14);


In [ ]:
# the price distribution of the invoices in right skewed as we see below, if we want to display the maximum values we will end up with no insights on the graph.

plt.figure(figsize=(14,8))

bins= np.linspace(start=1400, stop=max_order_payment_value, num=60)
plt.hist(x=np.asarray(master_df.groupby('customer_id').sum().reset_index()['payment_value']), bins=bins,\
         color='skyblue');

plt.xlim(0,110000);
plt.title('payment value distribution per customer'.title()+'\n'+'Between 1400LCU and 110,000LCU' , fontsize=20);
plt.ylabel('count'.title(), fontsize=14);
plt.xlabel('payment value'.title(), fontsize=14);


In [ ]:
customer_state_count_df = master_df.groupby(['customer_id','customer_state']).count().reset_index()

In [ ]:
plt.figure(figsize=(14,8))
sns.countplot(customer_state_count_df.customer_state, order=customers_dataset_df.customer_state.value_counts().index);

plt.title('What is the most frequent customer state?'.title(), fontsize=20);
plt.ylabel('count'.title(), fontsize=14);
plt.xlabel('customer state'.title(), fontsize=14);

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
# adding the translated product name category to the data_frame to use it in the following plots.
master_df_with_product_categories = master_df.merge(product_category_name_translation_df, how='left', on='product_category_name');

In [ ]:
plt.figure(figsize=(14,8))

sns.countplot(master_df_with_product_categories.product_category_name_english,#
              order=master_df_with_product_categories.product_category_name_english.value_counts().index)

plt.title('What is the most in demand product category?'.title(), fontsize=20);
plt.ylabel('count'.title(), fontsize=14);
plt.xlabel('product category'.title(), fontsize=14);

plt.xticks(rotation=90, fontsize=10);
plt.yticks(fontsize=12)
plt.show()

## Bivariable plots

* Mean score vs products category (Bar plot)
* Mean score vs city (Bar plot)
* Order Price vs Freight Price (Scatter plot)

In [ ]:
mean_score_vs_cat_df = master_df_with_product_categories.groupby(['product_category_name_english']).mean()['review_score'].reset_index()

In [ ]:
# The on average the best product category rating wise is that of cds_dvds_musicals, and the securit_and_services seems to have
# the lowest rating.

plt.figure(figsize=(14,8))

sns.barplot(data = master_df_with_product_categories, x='product_category_name_english', y='review_score',\
           order = master_df_with_product_categories.groupby('product_category_name_english').mean()\
            .reset_index().sort_values('review_score', ascending=False)['product_category_name_english'].values,
           errcolor = 'grey');

plt.title('what is the most highly rated product category?'.title(), fontsize=20);
plt.ylabel('Average Review Score'.title(), fontsize=14);
plt.xlabel('Product Category'.title(), fontsize=14);

plt.xticks(rotation=90, fontsize= 10);
plt.yticks(fontsize=12)
plt.show()

In [ ]:
# The nicest state ? AP: macapa laranjal do jari

plt.figure(figsize=(14,8))

sns.barplot(data = master_df_with_product_categories, x='customer_state', y='review_score',\
           order = master_df_with_product_categories.groupby('customer_state').mean()\
            .reset_index().sort_values('review_score', ascending=False)['customer_state'].values,
           errcolor = 'grey');


plt.title('what is the nicest customer state?'.title(), fontsize=20);
plt.ylabel('Average Review Score'.title(), fontsize=14);
plt.xlabel('customer state'.title(), fontsize=14);

plt.xticks(rotation=0, fontsize= 12);
plt.yticks(fontsize=12)
plt.show()

In [ ]:
# is there any relationship between the price of the order and the price of the freight? no

payment_vs_freight_df = master_df_with_product_categories.groupby('customer_id').sum().reset_index()[['payment_value','freight_value']]


In [ ]:
plt.figure(figsize=(14,8))
sns.regplot(data=payment_vs_freight_df, x=payment_vs_freight_df.payment_value, y=payment_vs_freight_df.freight_value,
            scatter_kws={'alpha':0.7, 'color':'skyblue'}, line_kws={'color':'red'});
plt.xlim(0,20000);

plt.title('Payment Vs. Freight value'.title(), fontsize=20);
plt.ylabel('Freight value'.title(), fontsize=14);
plt.xlabel('Payment Value'.title(), fontsize=14);

plt.xticks(rotation=0, fontsize= 12);
plt.yticks(fontsize=12)
plt.show()

## Multivariable Plots
* Payment value vs. month vs. year

In [ ]:
master_df_with_product_categories['year'] = pd.DatetimeIndex(master_df_with_product_categories['order_purchase_timestamp']).year

In [ ]:
master_df_with_product_categories['month'] = pd.DatetimeIndex(master_df_with_product_categories['order_purchase_timestamp']).month 

In [ ]:
master_df_with_product_categories.groupby(['year','month']).sum()['payment_value']

In [ ]:
multivariable_plot_df = master_df_with_product_categories.groupby(['year','month']).sum()['payment_value'].reset_index()
multivariable_plot_df


In [ ]:
g = sns.FacetGrid(data=multivariable_plot_df, row = 'year', height=7, aspect=2.5, sharey=True)

g = g.map(plt.bar, 'month', 'payment_value', color='skyblue')
g.set_titles( size=20);
g.set_ylabels('Payment Value'.title(), size=14);
g.set_xlabels('months'.title(), size=14);



# Machine Learning Applications
* Clustering customers based on customers data
* Customer Life Time Value prediction

## Clustering Customers Based On Review Rating and Purchase Power 
## (Unsupervised Learning)

In [ ]:
clustering_customers_df = master_df_with_product_categories[['customer_unique_id','customer_id','payment_value','review_score']]\
.groupby('customer_unique_id').agg({'payment_value': 'sum', 'review_score': 'mean', 'customer_id':'count'}).reset_index()

In [ ]:
clustering_customers_df.describe()

In [ ]:
# lets check the scatter
plt.figure(figsize=(14,8));

plt.scatter( clustering_customers_df.payment_value, clustering_customers_df.review_score, color='skyblue');
plt.xlim(0,35000);


plt.title('Payment value vs. review score (Not Scaled)'.title(), fontsize=20);
plt.ylabel('review score'.title(), fontsize=14);
plt.xlabel('Payment value'.title(), fontsize=14);

plt.xticks(rotation=0, fontsize= 12);
plt.yticks(fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(14,8))

bins= np.linspace(start=0, stop=1400, num=50)
plt.hist(x=np.asarray(clustering_customers_df.payment_value), bins=bins, color='skyblue');

plt.title('Payment value distribution'.title(), fontsize=20);
plt.ylabel('count'.title(), fontsize=14);
plt.xlabel('Payment value'.title(), fontsize=14);

plt.xticks(rotation=0, fontsize= 12);
plt.yticks(fontsize=12)
plt.show()

In [ ]:
clustering_customers_df.payment_value.describe()

In [ ]:
plt.figure(figsize=(14,8))
sns.scatterplot(x=scale(clustering_customers_df.payment_value), y=scale(clustering_customers_df.review_score), alpha=0.8,\
                x_jitter=0.5, y_jitter=0.5, s=80, data=clustering_customers_df, color='skyblue');

plt.title('Payment value vs. review score (scaled)'.title(), fontsize=20);
plt.ylabel('review score'.title(), fontsize=14);
plt.xlabel('Payment value'.title(), fontsize=14);

plt.xticks(rotation=0, fontsize= 12);
plt.yticks(fontsize=12)
plt.show()

In [ ]:
scale(clustering_customers_df.review_score).mean()

In [ ]:
clustering_customers_df['scaled_payment_value'] = scale(clustering_customers_df.payment_value)

In [ ]:
clustering_customers_df['scaled_review_score'] = scale(clustering_customers_df.review_score)

In [ ]:
clustering_customers_df[['scaled_payment_value','scaled_review_score']]

In [ ]:
# KMeans
kmeans = KMeans(4)
kmeans.fit(clustering_customers_df[['scaled_payment_value','scaled_review_score']])
clusters = clustering_customers_df[['scaled_payment_value','scaled_review_score']].copy()
clustering_customers_df['cluster_pred'] = kmeans.fit_predict(clusters)

In [ ]:
clustering_customers_df

In [ ]:
plt.figure(figsize=(14,8))
ax = sns.scatterplot(x=scale(clustering_customers_df.payment_value), y=scale(clustering_customers_df.review_score), alpha=0.8,\
                x_jitter=0.5, y_jitter=0.5, hue='cluster_pred', data=clustering_customers_df,\
               s=80, palette=sns.color_palette("hls", 4));


plt.setp(ax.get_legend().get_title(), fontsize='25')
plt.setp(ax.get_legend().get_texts(), fontsize='20')
plt.title('Clustering Customers'+'\n'+'Payment vaue vs. review score (4 clusters scaled)'.title(), fontsize=20);
plt.ylabel('review score'.title(), fontsize=14);
plt.xlabel('Payment value'.title(), fontsize=14);

plt.xticks(rotation=0, fontsize= 12);
plt.yticks(fontsize=12)
plt.show()

## Implementing the Elbow Method to choose the number of clusters

In [ ]:
# The Elbow method is mainly about plotting the Within Cluster Standard Error (WCSS) vs the Number of Clusters used.
# And we will be checking for a number of clusters that forms an elbow shape, meaning that the slope of our line changes highly when it
# goes into the number of clusters,(Slope_line_before_x_clusters+point != Slope_line_after_x_clusters_point)

wcss = []
for i in range(1,10):
    kmeans = KMeans(i)
    kmeans.fit(clustering_customers_df[['scaled_payment_value','scaled_review_score']])
    wcss.append(kmeans.inertia_)

In [ ]:
# 3 seems to have a good change in wcss, we will try it.
plt.figure(figsize=(14,8))

plt.plot(range(1,10), wcss, linewidth=4);
plt.xlabel('Number of clusters'.title())
plt.ylabel('wcss'.title())

plt.title('WCSS vs. Number of Clusters', fontsize=20);
plt.ylabel('WCSS', fontsize=14);
plt.xlabel('Number of Clusters'.title(), fontsize=14);

plt.xticks(rotation=0, fontsize= 12);
plt.yticks(fontsize=12)
plt.show()

In [ ]:
kmeans_new = KMeans(3)
kmeans_new.fit(clustering_customers_df[['scaled_payment_value','scaled_review_score']])
clustering_customers_df['clusters_new_3'] = kmeans_new.fit_predict(clustering_customers_df[['scaled_payment_value','scaled_review_score']])
clustering_customers_df

In [ ]:
plt.figure(figsize=(14,8))
ax= sns.scatterplot(x=scale(clustering_customers_df.payment_value), y=scale(clustering_customers_df.review_score), alpha=0.8,\
                x_jitter=0.5, y_jitter=0.5, hue='clusters_new_3', data=clustering_customers_df,\
               s=80, palette=sns.color_palette("hls", 3));


plt.setp(ax.get_legend().get_title(), fontsize='25')
plt.setp(ax.get_legend().get_texts(), fontsize='20')
plt.title('Clustering Customers'+'\n'+'Payment value vs. review score (3 clusters scaled)'.title(), fontsize=20);
plt.ylabel('review score'.title(), fontsize=14);
plt.xlabel('Payment value'.title(), fontsize=14);

plt.xticks(rotation=0, fontsize= 12);
plt.yticks(fontsize=12)
plt.show()

In [ ]:
# from a business point of view I believe the last cluster seem to be a group of outliers,
#they have alot of purchase power, some of them are satistfied some are not, if we isolate this group, it is better to use 5 
# clusters is order to have the for typical clusters: Fans, supporters, Roamers, Alienated, in addition to our case, the group of outliers

kmeans_new_5 = KMeans(5)
kmeans_new_5.fit(clustering_customers_df[['scaled_payment_value','scaled_review_score']])
clustering_customers_df['clusters_new_5'] = kmeans_new_5.fit_predict(clustering_customers_df[['scaled_payment_value','scaled_review_score']])
clustering_customers_df

In [ ]:
# as we see no real added value happened to our clustering of 5, which validate the Elbow method, since on the graph we do not see
# any real change in the WCSS with the increase of number of clusters.

plt.figure(figsize=(14,8))

ax = sns.scatterplot(x=scale(clustering_customers_df.payment_value), y=scale(clustering_customers_df.review_score), alpha=0.8,\
                x_jitter=0.5, y_jitter=0.5, hue='clusters_new_5', data=clustering_customers_df,\
               s=80, palette=sns.color_palette("husl", 5));


plt.setp(ax.get_legend().get_title(), fontsize='25')
plt.setp(ax.get_legend().get_texts(), fontsize='20')
plt.title('Clustering Customers'+'\n'+'Payment value vs. review score (5 clusters scaled)'.title(), fontsize=20);
plt.ylabel('review score'.title(), fontsize=14);
plt.xlabel('Payment value'.title(), fontsize=14);

plt.xticks(rotation=0, fontsize= 12);
plt.yticks(fontsize=12)
plt.show()